# Fundamentals of Network Science

**Complex Social Systems, 2025** </br>
*Onur Akman, Lab 3, 26/03/2025*

Colab: https://colab.research.google.com/drive/1MZzsrD5TsyRgb9toT-qhwTb0N0Ib_jt0?usp=sharing

In [ ]:
import itertools
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import random
import scipy

from collections import Counter

random.seed(42)
np.random.seed(42)

# 1) Refresher: Graphs

In [ ]:
G = nx.Graph([(1, 2), (2, 3), (3, 4)])
print(f'\
    Edges: {G.edges}, \n\
    Nodes: {G.nodes}, \n\
    Degrees: {G.degree}, \n\
    Connections (as numpy array):\n{nx.to_numpy_array(G)} \
    ')

In [ ]:
layout = nx.circular_layout(G)
fig, ax = plt.subplots(figsize=(4, 4))
nx.draw_networkx(G, pos=layout)
ax.axis("off")
plt.show()
plt.close()

## 1.2) Defining Random Networks

$G_{n, p}$ Model: Each pair of `n` labeled nodes is connected with probability `p`. </br>
In NetworkX, this basic family of random graphs is implemented as `fast_gnp_random_graph`. </br>
Also known as an **Erdős-Rényi** graph or a **binomial graph**.

In [ ]:
G = nx.fast_gnp_random_graph(10, 0.3, seed=42)
degree_sequence = sorted([d for _, d in G.degree()])
degree_counter = Counter(degree_sequence)
deg, cnt = zip(*degree_counter.items())

print(f"Average degree: {sum(degree_sequence) / len(degree_sequence)}")

fig, ax = plt.subplots(figsize=(4, 3))
plt.bar(deg, cnt)
plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Degree")
plt.tight_layout()
plt.show()
plt.close()

---

### Exercise 1: Random graphs

>In a binomial random graph, investigate how does an average **node degree** changes with increasing: <br>
>    a) number of nodes; <br>
>    b) increasing probability of linkage.

In [ ]:
# Solution

---
# 2) Retrieve networks

In [ ]:
facebook_df = pd.read_csv(
    "https://snap.stanford.edu/data/facebook_combined.txt.gz",
    compression="gzip",
    sep=" ",
    names=["start_node", "end_node"],
)

twitter_df = pd.read_csv(
    "https://snap.stanford.edu/data/twitter_combined.txt.gz",
    compression="gzip",
    sep=" ",
    names=["start_node", "end_node"]
)

In [ ]:
print("--- Facebook data ---\n", facebook_df.sample(5, random_state=42))
print(f"Facebook data shape: {facebook_df.shape}")
print("\n--- Twitter data ---\n", twitter_df.sample(5, random_state=42))
print(f"Twitter data shape: {twitter_df.shape}")

**More insights**

In [ ]:
facebook_G = nx.from_pandas_edgelist(facebook_df, "start_node", "end_node")
print("-- Facebook Graph --")
print(f"Graph type: {type(facebook_G)}")
print(f"Number of nodes: {facebook_G.number_of_nodes()}")
print(f"Number of edges: {facebook_G.number_of_edges()}")
print(f"Average degree: {sum(dict(facebook_G.degree()).values()) / facebook_G.number_of_nodes()}")

twitter_G = nx.from_pandas_edgelist(twitter_df, "start_node", "end_node")
print("\n-- Twitter Graph --")
print(f"Graph type: {type(twitter_G)}")
print(f"Number of nodes: {twitter_G.number_of_nodes()}")
print(f"Number of edges: {twitter_G.number_of_edges()}")
print(f"Average degree: {sum(dict(twitter_G.degree()).values()) / twitter_G.number_of_nodes()}")


In [ ]:
def random_walk_sample(G, target_size, seed=42):
    random.seed(seed)
    start_node = random.choice(list(G.nodes))
    visited = {start_node}
    current = start_node
    neighbors = []

    while len(visited) < target_size:
        neighbors = [n for n in  neighbors + list(G.neighbors(current)) if n not in visited]
        neighbors = list(set(neighbors))
        if not neighbors:
            start_node = random.choice(list(G.nodes))
            visited = {start_node}
            current = start_node
            continue
        current = random.choice(neighbors)
        visited.add(current)
    return G.subgraph(visited).copy()

twitter_G = random_walk_sample(twitter_G, facebook_G.number_of_nodes(), seed=42)
print(f"Number of nodes: {twitter_G.number_of_nodes()}")
print(f"Number of edges: {twitter_G.number_of_edges()}")
print(f"Average degree: {sum(dict(twitter_G.degree()).values()) / twitter_G.number_of_nodes()}")

**Have a look at our of the networks using spring layout**

In [ ]:
pos = nx.spring_layout(facebook_G, iterations=17, seed=42)

fig, ax = plt.subplots(figsize=(8, 6))
ax.axis("off")
nx.draw_networkx(facebook_G, pos=pos, ax=ax, node_size=10, with_labels=False, width=0.15)
plt.show()
plt.close()

In [ ]:
pos = nx.spring_layout(twitter_G, iterations=17, seed=42) # Try also Twitter

fig, ax = plt.subplots(figsize=(8, 6))
ax.axis("off")
nx.draw_networkx(twitter_G, pos=pos, ax=ax, node_size=10, with_labels=False, width=0.15)
plt.show()
plt.close()

---
### Exercise 2: Six handshakes

<img src="https://people.com/thmb/Q_qTsGov1TBp3lCJr0lI0sxP6VA=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc():focal(958x681:960x683)/Footloose-Kevin-Bacon-061624-01-20ddcdcf0d8b42f89e9ca908cc63edb1.jpg" align="right" width="20%"/>

There is a famous idea called [**six handshakes rule**](https://en.wikipedia.org/wiki/Six_degrees_of_separation).

**Six degrees of separation** is the idea that all people are six or fewer social connections away from each other. </br>
As a result, a chain of *friend of a friend* statements can connect any two people in a maximum of **six steps**. </br>
It is also known as the **six handshakes rule**.

> Compute the average distance between pairs of nodes in the graph using [NetworkX shortest path functions](https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html).

In [ ]:
# Solution

---
# 3) Communities

- **Communities** in networks are **groups of nodes** that are more **densely connected** to each other than to the rest of the network. 
- The **Girvan-Newman** algorithm is a method for identifying communities within networks by **iteratively removing edges with high betweenness centrality**.
- By targeting and eliminating these critical connections—often the bridges between clusters—the algorithm gradually reveals the network’s underlying community structure, offering valuable insights into how nodes group together.
- **Betweenness centrality** measures **how often a node appears on the shortest paths between other nodes in the network**. It is calculated as:

$$C_B(v) = \sum_{s \neq v \neq t} \frac{\sigma_{st}(v)}{\sigma_{st}},$$

where $\sigma_{st}$ is the total number of shortest paths from node $s$ to node $t$, and $\sigma_{st}(v)$ is the number of those paths that pass through $v$.

In [ ]:
def draw(G, edges_to_highlight, colour_map=None, iter_step=-1, func=nx.shell_layout):
    try:
        layout = func(G, seed=42)
    except:
        layout = func(G)
    fig, ax = plt.subplots(figsize=(6, 6))
    if colour_map is not None:
        nx.draw_networkx(G, node_color=colour_map, pos=layout, node_size=40, with_labels=False, width=0.15)
        nx.draw_networkx_edges(G, pos=layout, edgelist=edges_to_highlight, width=2, edge_color=(1, 0, 0, 1))
    else:
        nx.draw_networkx(G, pos=layout, node_size=40, with_labels=False, width=0.15)
    ax.axis("off")
    if iter_step != -1:
        plt.title(f'Step: {iter_step}')
    plt.show()
    plt.close()

The `karate_club_graph` in NetworkX is a well-known social network representing the interactions among members of a karate club.

In [ ]:
G = nx.karate_club_graph()
print(type(G))
draw(G, [], iter_step=None, func=nx.spring_layout)

In [ ]:
target_communities = 3
num_conn_comp = 1
#draw(G, [], None, 1, func=nx.spring_layout)

# Copy of G for modification
G_temp = G.copy()

# To keep track of removed edges
removed_edges = list()

# Use a consistent layout for all plots
pos = nx.spring_layout(G, seed=42)

# List of distinct colors
base_colors = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.BASE_COLORS.values())

while True:
    
    if num_conn_comp == target_communities:
        break
    
    # 1. Compute edge betweenness centrality
    edge_betweenness = nx.edge_betweenness_centrality(G_temp)
    
    # 2. Remove the edge with highest betweenness
    edge_to_remove = max(edge_betweenness, key=edge_betweenness.get)
    G_temp.remove_edge(*edge_to_remove)
    removed_edges.append(edge_to_remove)
    
    # 3. Find connected components as communities
    communities = list(nx.connected_components(G_temp))
    
    # 4. Map node to community color
    color_map = [''] * G.number_of_nodes()
    for color_index, community in enumerate(communities):
        for node in community:
            color_map[node] = list(mcolors.BASE_COLORS.keys())[color_index % len(mcolors.BASE_COLORS)]
            
    if len(communities) > num_conn_comp:
        #draw(G, removed_edges, color_map, len(communities), func=nx.spring_layout)
        num_conn_comp = len(communities)
        
nx.draw_networkx(G_temp, pos=pos, node_color=color_map, node_size=40, with_labels=False, width=0.15)
plt.title("Final Community Structure (Between edges ommitted)")
plt.show()
plt.close()

---
### Exercise 3: Finding communities using Girvan Newman

NetworkX provides an implementation of the Girvan Newman method. </br>
> Use NetworkX's `nx.community.girvan_newman` method (find [here](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.centrality.girvan_newman.html)) to regenerate the communities we found above.

In [ ]:
# Solution

---
### Exercise 4: Impact of number of communities

> Using **Girvan-Newman method**, create loop where you measure the **diameter and an average distance within communites** in a **facebook_G subgraph** (100 nodes) for increasing number of communities (up to $10^{th}$ step).

**Hint:** The diameter of a graph is the length of the shortest path between the most distanced nodes (see `nx.diameter`).

In [ ]:
G = random_walk_sample(facebook_G, 100, seed=42)

# Solution

---

# 4) Centralities: PageRank

- PageRank centrality is based on Google's algorithm used to evaluate the importance or relevance of nodes in a network. (See [here](https://en.wikipedia.org/wiki/PageRank))
- It assigns **a score to each node** based on the **number and quality of links (or edges) pointing to it**. 
- It is used to rank web pages in search engines, but it can also be applied to other types of networks to identify key players or important nodes.
- It is defined by the **recursive** formula:

$$
PR(i) = \frac{1-d}{N} + d \sum_{j \in M(i)} \frac{PR(j)}{L(j)},
$$

- where:
    - $PR(i)$ is the PageRank of node $i$, 
    - $d$ is the damping factor (typically set to 0.85), 
    - $N$ is the total number of nodes, 
    - $M(i)$ is the set of nodes linking to $i$, and 
    - $L(j)$ is the number of outbound links from node $j$.
- Breaking it down:
    - $\frac{1-d}{N}$ assigns a baseline score to every node, 
    - The damping factor $d$ represents the probability that **a random surfer continues following links rather than jumping to a random node**,
    - The sum $\sum_{j \in M(i)} \frac{PR(j)}{L(j)}$ aggregates the contribution of each node $j$ that points to node $i$.
- Intuition:
    - This formulation captures that **a node’s importance depends not just on the number of incoming links, but on the quality of those links**.
    - A node gains more importance if it is **linked by other important nodes that do not spread their influence** too thinly among many targets.

<div align="center">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/PageRanks-Example.svg/1920px-PageRanks-Example.svg.png" alt="Source: https://en.wikipedia.org/wiki/PageRank" width="400" />
</div>

In [ ]:
G = nx.karate_club_graph()

pagerank = nx.pagerank(G, alpha=0.85)

pos = nx.spring_layout(G, seed=42, k=2)


node_sizes = [10000 * pagerank[node] for node in G.nodes()]
node_colors = [pagerank[node] for node in G.nodes()]


plt.figure(figsize=(8, 8))
nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes, 
                               node_color=node_colors, cmap=plt.cm.magma)
nx.draw_networkx_edges(G, pos, alpha=0.5, arrows=True, width=2, arrowsize=20, arrowstyle='-|>')
plt.colorbar(nodes, label='PageRank Centrality')

plt.title("Graph Visualization Based on PageRank Centrality")
plt.axis('off')
plt.show()

---
### Exercise 5: Impact of number of the damping factor

*The damping factor is a parameter that represents the probability that a user will follow a link on a page rather than randomly jumping to any other page.*

> Investigate the influence of the damping factor in social media networks. With decreasingly probability of random jumps, what kind of changes do we see in PageRank centralities?

In [ ]:
# Start by recreating our graphs, this time as ! directed graphs !

facebook_G = nx.from_pandas_edgelist(facebook_df, "start_node", "end_node", create_using=nx.DiGraph())
twitter_G = nx.from_pandas_edgelist(twitter_df, "start_node", "end_node", create_using=nx.DiGraph())
#twitter_G = random_walk_sample(twitter_G, facebook_G.number_of_nodes(), seed=42)

In [ ]:
# Solution

---
### Exercise 6: Network communities and local hubs

> Examine how the PageRank centralities evolve within the community subgraphs of a Twitter network as the number of communities increases. (up to 4 iterations)

In [ ]:
# Solution